<a href="https://colab.research.google.com/github/satyam-52/speech-to-text/blob/main/speech_to_text_whisper_hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch torchaudio accelerate googletrans==4.0.0rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━

In [2]:
import torch
from transformers import pipeline
from google.colab import files
from googletrans import Translator
import IPython.display as ipd
import os

In [3]:
def initialize_models():
    """Initialize transcription model and Google Translator"""
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # Transcription model
    print("Loading Whisper Hindi Large-v2 model...")
    transcribe = pipeline(
        task="automatic-speech-recognition",
        model="vasista22/whisper-hindi-large-v2",
        chunk_length_s=30,
        device=device
    )

    transcribe.model.config.forced_decoder_ids = transcribe.tokenizer.get_decoder_prompt_ids(
        language="hi",
        task="transcribe"
    )

    # Google Translator
    print("Initializing Google Translator...")
    translator = Translator()

    print("Models loaded successfully!")
    return transcribe, translator

In [4]:
def transcribe_and_translate_google(audio_path, transcribe_model, translator):
    """
    Transcribe Hindi audio and translate to English using Google Translate
    """
    try:
        # Transcribe
        print("Transcribing audio...")
        result = transcribe_model(audio_path)
        hindi_text = result["text"]

        # Translate using Google Translate
        print("Translating to English using Google Translate...")
        translation = translator.translate(hindi_text, src='hi', dest='en')
        english_text = translation.text

        return hindi_text, english_text

    except Exception as e:
        print(f"Error: {e}")
        return None, None

In [5]:
# Test the translation with your example
def test_translation():
    translator = Translator()
    test_text = "मेरा नाम सत्यम है"
    result = translator.translate(test_text, src='hi', dest='en')
    print(f"Test Hindi: {test_text}")
    print(f"Test English: {result.text}")

In [6]:
# Run test first
print("Testing translation accuracy:")
test_translation()

# Initialize models
transcriber, translator = initialize_models()

# Upload and process audio
print("\nPlease upload your Hindi audio file:")
uploaded = files.upload()
audio_file = list(uploaded.keys())[0]

# Process the audio
hindi_transcription, english_translation = transcribe_and_translate_google(
    audio_file, transcriber, translator
)

if hindi_transcription and english_translation:
    print("\n" + "="*60)
    print("RESULTS:")
    print("="*60)
    print("📝 HINDI TRANSCRIPTION:")
    print(hindi_transcription)
    print("\n🔄 ENGLISH TRANSLATION (Google Translate):")
    print(english_translation)
    print("="*60)
else:
    print("Processing failed!")


# Save Hindi transcription
hindi_filename = f"{audio_file}_hindi_transcription.txt"
with open(hindi_filename, 'w', encoding='utf-8') as f:
    f.write(hindi_transcription)

# Save English translation
english_filename = f"{audio_file}_english_translation.txt"
with open(english_filename, 'w', encoding='utf-8') as f:
    f.write(english_translation)

# Save combined results
combined_filename = f"{audio_file}_transcription_and_translation.txt"
with open(combined_filename, 'w', encoding='utf-8') as f:
    f.write("HINDI TRANSCRIPTION:\n")
    f.write("=" * 30 + "\n")
    f.write(hindi_transcription + "\n\n")
    f.write("ENGLISH TRANSLATION:\n")
    f.write("=" * 30 + "\n")
    f.write(english_translation + "\n")

print(f"\nFiles saved:")
print(f"- Hindi transcription: {hindi_filename}")
print(f"- English translation: {english_filename}")
print(f"- Combined file: {combined_filename}")

# Download the files
files.download(hindi_filename)
files.download(english_filename)
files.download(combined_filename)


Testing translation accuracy:
Test Hindi: मेरा नाम सत्यम है
Test English: My name is Satyam
Using device: cuda:0
Loading Whisper Hindi Large-v2 model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>